In [0]:
import numpy as np
import torch
import pickle
import torch.utils.data as data_utils
import pandas as pd
import torch.nn as nn
import time


from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms

In [0]:
with open('/content/gdrive/My Drive/features_50.pkl', 'rb') as f:
  features = pickle.load(f)
  
with open('/content/gdrive/My Drive/folds_50.pkl', 'rb') as f:
  folds = pickle.load(f)
  
with open('/content/gdrive/My Drive/labels_50.pkl', 'rb') as f:
  labels = pickle.load(f)

## Numpy to Tensors

In [0]:
print(len(features), len(folds), len(labels))

3676 3676 3676


In [0]:
features[0].shape

(50, 50, 3)

In [0]:
features_arr = np.asarray(features, dtype=np.float32)
labels_arr = np.asarray(labels, dtype=np.int64)
folds_array = np.asarray(folds)

In [0]:
features_arr = np.swapaxes(features_arr, 1, 3)
features_arr = np.swapaxes(features_arr, 2, 3)

In [0]:
print(features_arr.shape, labels_arr.shape)

(3676, 3, 50, 50) (3676,)


In [0]:
unique_items, counts = np.unique(folds, return_counts=True)
unique_items, counts

(array(['fold1', 'fold2', 'fold3', 'fold4'], dtype='<U5'),
 array([873, 888, 925, 990]))

In [0]:
sum1 = counts[0]
sum12 = counts[0] + counts[1]
sum23 = sum12 + counts[2]
sum34 = sum23 + counts[3]

In [0]:
y_te1 = labels_arr[0:sum1]
y_tr1 = labels_arr[sum1:]
X_te1 = features_arr[0:sum1, :, :, :]
X_tr1 = features_arr[sum1:, :, :, :]
folds_1 = np.delete(folds_array, list(range(0, sum1)))

In [0]:
y_te2 = labels_arr[sum1:sum12]
y_tr2 = np.concatenate([labels_arr[0:sum1], labels_arr[sum12:]])
X_te2 = features_arr[sum1:sum12, :, :, :]
X_tr2 = np.delete(features_arr, slice(sum1, sum12), axis=0)
folds_2 = np.delete(folds_array, list(range(sum1, sum12)))

In [0]:
y_te3 = labels_arr[sum12:sum23]
y_tr3 = np.concatenate([labels_arr[0:sum12], labels_arr[sum23:]])
X_te3 = features_arr[sum12:sum23, :, :, :]
X_tr3 = np.delete(features_arr, slice(sum12,sum23), axis=0)
folds_3 = np.delete(folds_array, list(range(sum12,sum23)))

In [0]:
y_te4 = labels_arr[sum23:sum34]
y_tr4 = np.concatenate([labels_arr[0:sum23], labels_arr[sum34:]])
X_te4 = features_arr[sum23:sum34, :, :, :]
X_tr4 = np.delete(features_arr, slice(sum23,sum34), axis=0)
folds_4 = np.delete(folds_array, list(range(sum23,sum34)))

In [0]:
group_list = [folds_1, folds_2, folds_3, folds_4]
X_train_list = [X_tr1, X_tr2, X_tr3, X_tr4]
y_train_list = [y_tr1, y_tr2, y_tr3, y_tr4]
X_test_list = [X_te1, X_te2, X_te3, X_te4]
y_test_list = [y_te1, y_te2, y_te3, y_te4]

## Model


In [0]:
import torch
from torch.utils.data import Dataset, DataLoader

import numpy as np


#Define transformations for the training set, flip the images randomly, crop out and apply mean and std normalization
train_transformations = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop(32,padding=4),
#     transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])




#Define transformations for the test set
test_transformations = transforms.Compose([
#     transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))

])



class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)

train_loaders = []
test_loaders = []
for i in range(4):
  
  numpy_data_tr = X_train_list[i]
  numpy_target_tr = y_train_list[i]
  
  
  dataset_tr = MyDataset(numpy_data_tr, numpy_target_tr, transform=train_transformations)
  loader_tr = DataLoader(
      dataset_tr,
      batch_size=20,
      shuffle=False,
      num_workers=2,
      pin_memory=torch.cuda.is_available()
  )
  
  numpy_data_te = X_test_list[i]
  numpy_target_te = y_test_list[i]
  
  dataset_te = MyDataset(numpy_data_te, numpy_target_te, transform=test_transformations)
  loader_te = DataLoader(
      dataset_te,
      batch_size=20,
      shuffle=False,
      num_workers=2,
      pin_memory=torch.cuda.is_available()
  )
  
  train_loaders.append(loader_tr)
  test_loaders.append(loader_te)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
#Import needed packages
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim import SGD
from torch.autograd import Variable
import numpy as np
from sklearn.metrics import f1_score



class SimpleNet(nn.Module):
    def __init__(self, num_classes=10):
        
        super(SimpleNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride = 2)

        self.dropout1 = nn.Dropout(.4)

        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        
        self.dropout2 = nn.Dropout(.3)

        
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride = 2)
        
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=36, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
#         self.pool3 = nn.MaxPool2d(kernel_size=2, stride = 2)

        self.dropout3 = nn.Dropout(.3)
                               
        self.fc = nn.Linear(in_features=50 * 50 * 9, out_features=num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.relu1(output)
        output = self.dropout1(output)

#         output = self.pool1(output)
        
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.dropout2(output)

        output = self.pool2(output)


        output = self.conv3(output)
        output = self.relu3(output)
#         output = self.pool3(output)
        
        output = self.dropout3(output)
        output = output.view(-1, 50 * 50 * 9)

        output = self.fc(output)

        return output



log_freq = 40

cuda_avail = torch.cuda.is_available()

model = SimpleNet()

if cuda_avail:
    model.cuda()

optimizer = Adam(model.parameters(), lr=0.0001)
# optimizer = SGD(model.parameters(), lr = .005, momentum=.9)
loss_fn = nn.CrossEntropyLoss()



def save_models(epoch):
    torch.save(model.state_dict(), "conv_{}.model".format(epoch))
    print("Checkpoint saved")

def test():
    model.eval()
    test_acc = 0.0
    running_loss = 0.0
    running_accuracy = 0.0
    for i, (images, labels) in enumerate(test_loaders[1]):
      
        if cuda_avail:
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

        output = model(images)

        loss = loss_fn(output, labels)
        running_loss += loss.item()
        y_pred = output.argmax(dim=-1)  
        
        running_accuracy += f1_score(labels.cpu(), y_pred.cpu(), average='macro')
        
    average_loss = running_loss / len(test_loaders[1])
    average_accuracy = running_accuracy / (len(test_loaders[1]))
    
    
    print(f'Val Loss: {average_loss:.5f} Val Accuracy: {average_accuracy:.5f}\n')
    test_acc = test_acc / len(test_loaders[1])
    return average_accuracy

def train(num_epochs):
    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_accuracy = 0.0
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loaders[1]):
            
            if cuda_avail:
                images = Variable(images.cuda())
                labels = Variable(labels.cuda())

            output = model(images)
            loss = loss_fn(output,labels)
            optimizer.zero_grad()

            loss.backward()
            optimizer.step()
           
            running_loss += loss.item()
            y_pred = output.argmax(dim=-1)
            running_accuracy += f1_score(labels.cpu(), y_pred.cpu(), average='macro')
            if i % log_freq == log_freq - 1:
              average_loss = running_loss / len(train_loaders[1])
              average_accuracy = running_accuracy / len(train_loaders[1])
              print(f'Mini-batch: {i + 1}/{len(train_loaders[1])} '
                  f'Loss: {average_loss:.5f} Accuracy: {average_accuracy:.5f}')
              running_loss = 0.0
              running_accuracy = 0.0
              

        average_accuracy = test()

        if average_accuracy > best_acc:
            save_models(epoch)
            best_acc = average_accuracy

if __name__ == "__main__":
    train(200)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Mini-batch: 40/140 Loss: 12.26865 Accuracy: 0.01533
Mini-batch: 80/140 Loss: 6.78124 Accuracy: 0.02353
Mini-batch: 120/140 Loss: 4.68065 Accuracy: 0.02550
Val Loss: 2.98574 Val Accuracy: 0.10422

Checkpoint saved
Mini-batch: 40/140 Loss: 3.14560 Accuracy: 0.02785
Mini-batch: 80/140 Loss: 2.45485 Accuracy: 0.02884
Mini-batch: 120/140 Loss: 2.05469 Accuracy: 0.03329
Val Loss: 2.52511 Val Accuracy: 0.10158

Mini-batch: 40/140 Loss: 1.49037 Accuracy: 0.03958
Mini-batch: 80/140 Loss: 1.21201 Accuracy: 0.03608
Mini-batch: 120/140 Loss: 1.15099 Accuracy: 0.03489
Val Loss: 2.29319 Val Accuracy: 0.08022

Mini-batch: 40/140 Loss: 0.91104 Accuracy: 0.04346
Mini-batch: 80/140 Loss: 0.84320 Accuracy: 0.04544
Mini-batch: 120/140 Loss: 0.83427 Accuracy: 0.03788
Val Loss: 2.23469 Val Accuracy: 0.08700

Mini-batch: 40/140 Loss: 0.74739 Accuracy: 0.04146
Mini-batch: 80/140 Loss: 0.70168 Accuracy: 0.04258
Mini-batch: 120/140 Loss: 0.71556 Accuracy: 0.02936
Val Loss: 2.24502 Val Accuracy: 0.05806

Mini-ba